# File Downloading

In [1]:
import fcntl
import os
import errno
import requests
from sklearn.externals import joblib

In [9]:
def load_model(model_url, model_path):
    # Load users model if specified and exists
    # get exclusive lock on file and check to see if empty. If
    if os.path.isfile(model_path):
        with open(model_path, 'rb') as f:
            try:
                fcntl.flock(f, fcntl.LOCK_EX)
                if os.path.getsize(model_path) > 0:
                    clf = joblib.load(model_path)
            except IOError as e:
                if e.errno != errno.EAGAIN:
                    raise
            finally:
                fcntl.flock(f, fcntl.LOCK_UN)
    else:
        if os.path.dirname(model_path):
            if not os.path.isdir(os.path.dirname(model_path)):
                try:
                    os.makedirs(os.path.dirname(model_path))
                except OSError as exc: # Guard against rare condition
                    if exc.errno != errno.EEXIST:
                        raise
        with open(model_path, 'wb+') as f:
            try:
                fcntl.flock(f, fcntl.LOCK_EX | fcntl.LOCK_NB)
                if os.path.getsize(model_path) == 0:
                    r = requests.get(model_url, stream=True)
                    for chunk in r.iter_content(chunk_size=1024):
                        if chunk:  # filter out keep-alive new chunks
                            f.write(chunk)
            except BlockingIOError:
                fcntl.flock(f, fcntl.LOCK_EX)
            finally:
                fcntl.flock(f, fcntl.LOCK_UN)
        clf = joblib.load(model_path)
    return clf

In [11]:
load_model('https://s3-us-west-2.amazonaws.com/idmc-idetect/category_models/category.pkl',
          'model.pkl')

AttributeError: module '__main__' has no attribute 'Tokenizer'